In [1]:
import os
from pathlib import Path
import sys
curdir = Path(os.getcwd())
sys.path.append(str(curdir.parent.absolute()))
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from src.utils.data import read_fasta
from src.data.datasets import ProteinDataset

In [2]:
import torch
torch.stack([1,2,3])

TypeError: expected Tensor as element 0 in argument 0, but got int

In [2]:
train = read_fasta('../data/swissprot/proteinfer_splits/random/train_GO.fasta')
val = read_fasta('../data/swissprot/proteinfer_splits/random/dev_GO.fasta')
test = read_fasta('../data/swissprot/proteinfer_splits/random/test_GO.fasta')

In [3]:
train = [(k[0],j," ".join(k[1:])) for j,k in train]
test = [(k[0],j," ".join(k[1:])) for j,k in test]
val = [(k[0],j," ".join(k[1:])) for j,k in val]

df = train + val + test

In [4]:
df = pd.DataFrame(df,columns=['id','sequence','labels'])

In [5]:
num_sequences = len(df)
print('number of sequences:',num_sequences)

number of sequences: 522607


In [6]:
labels = Counter()

vocab = set()
amino_freq = Counter()
for idx,row in df.iterrows():
    sequence = row['sequence']
    row_labels = row['labels']
    aa_list = list(sequence)
    if row_labels =='':
        print(row['id'],row['labels'])
    vocab.update(aa_list)
    amino_freq.update(aa_list)
    labels.update(row_labels.split(" "))

In [7]:
print('# GO Terms:',len(labels.keys()))

# GO Terms: 32102


In [8]:
print('GO Terms distribution')
pd.Series(labels.values()).describe()

GO Terms distribution


count     32102.000000
mean        777.250545
std        9114.786603
min           1.000000
25%           4.000000
50%          17.000000
75%          84.000000
max      462356.000000
dtype: float64

In [9]:
print('Sequence length distribution')

df['sequence'].apply(len).describe()

Sequence length distribution


count    522607.000000
mean        368.042215
std         334.721845
min           2.000000
25%         179.000000
50%         303.000000
75%         456.000000
max       35213.000000
Name: sequence, dtype: float64

In [3]:
PD = ProteinDataset(data_path='../data/swissprot/proteinfer_splits/random/train_GO.fasta',
                    sequence_vocabulary_path='../data/vocabularies/amino_acid_vocab.json',
                    label_vocabulary_path='../data/vocabularies/GO_label_vocab.json')

In [4]:
len(PD),PD.get_max_seq_len()

(418015, 35213)

In [7]:
a = {'a':1,'b':2}
d = {'d':5,'c':4}

{**a,**d}

{'a': 1, 'b': 2, 'd': 5, 'c': 4}

In [6]:
import torch
from src.models.protein_encoders import Residual,ProteInfer
i=torch.ones((8,20,100))
#r=Residual(input_channels=20,kernel_size=9,dilation=9,bottleneck_factor=0.5,activation = torch.nn.ReLU)
prot_enc = ProteInfer(num_labels=32102,input_channels=20,output_channels=1100,kernel_size=9,activation=torch.nn.ReLU,dilation_base=3,num_resnet_blocks=5,bottleneck_factor=0.5)
seqs_lengths = torch.tensor([80,20,5,100,95,80,20,5])
o=prot_enc(i,seqs_lengths)
o.shape

torch.Size([8, 32102])

In [7]:
from src.utils.proteinfer import transfer_tf_weights_to_torch
transfer_tf_weights_to_torch(prot_enc,'../models/proteinfer/GO_model_weights.pkl')

Pytorch num variables: 74
Tensorflow num variables: 74


conv1.weight:torch.Size([1100, 20, 9]) <--> inferrer/conv1d/kernel:0:(1100, 20, 9)
conv1.bias:torch.Size([1100]) <--> inferrer/conv1d/bias:0:(1100,)
resnet_blocks.0.bn_activation_1.0.weight:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/gamma:0:(1100,)
resnet_blocks.0.bn_activation_1.0.bias:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/beta:0:(1100,)
resnet_blocks.0.bn_activation_1.0.running_mean:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/moving_mean:0:(1100,)
resnet_blocks.0.bn_activation_1.0.running_var:torch.Size([1100]) <--> inferrer/residual_block_0/batch_normalization/moving_variance:0:(1100,)
resnet_blocks.0.bn_activation_1.0.num_batches_tracked:torch.Size([]) <--> inferrer/residual_block_0/batch_normalization/num_batches_tracked:0:()
resnet_blocks.0.masked_conv1.weight:torch.Size([550, 1100, 9]) <--> inferrer/residual_block_0/conv1d/kernel:0:(550, 1

In [9]:
prot_enc

ProteInfer(
  (conv1): MaskedConv1D(20, 1100, kernel_size=(9,), stride=(1,), padding=same)
  (resnet_blocks): ModuleList(
    (0): Residual(
      (bn_activation_1): Sequential(
        (0): BatchNorm1d(1100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv1): MaskedConv1D(1100, 550, kernel_size=(9,), stride=(1,), padding=same)
      (bn_activation_2): Sequential(
        (0): BatchNorm1d(550, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv2): MaskedConv1D(550, 1100, kernel_size=(1,), stride=(1,), padding=same)
    )
    (1): Residual(
      (bn_activation_1): Sequential(
        (0): BatchNorm1d(1100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (masked_conv1): MaskedConv1D(1100, 550, kernel_size=(9,), stride=(1,), padding=same, dilation=(3,))
      (bn_activation_2): Sequential(
        (0): BatchNorm1d(550, 